### Perimeter Institute Nov 23rd
### 2018 computational physics course
# Exact Diagonalization
 
Guifre Vidal  

# IJulia nb4: quantum entanglement

In [ ]:
using PyPlot
using LinearAlgebra
using Arpack

In [ ]:
Psi = randn(6) + im*randn(6) # complex vector

In [ ]:
Psi'*Psi # already just a complex number! (not an array)

In [ ]:
Psi = Psi/sqrt(abs(Psi'*Psi))  # normalized vector
Psi'*Psi

In [ ]:
PsiAB = reshape(Psi, (3,2)) # bipartite vector

In [ ]:
rdmA = PsiAB*PsiAB' # reduced density matrix for part A
rdmB = PsiAB'*PsiAB # reduced density matrix for part B
(size(PsiAB), size(rdmA), size(rdmB))

In [ ]:
tr(rdmA) # checking that the trace is 1

### 1) Given a bipartite wavefunction $\Psi_{AB}$, compute the reduced density matrix $\rho_A \equiv \mbox{Tr}_B(\left|\Psi_{AB}\rangle \langle \Psi_{AB}\right|)$ 

In [ ]:
function compute_rdmA(PsiAB)
    @assert ndims(PsiAB) == 2 # did we input a matrix?
    @assert size(PsiAB,1) < 2^13 # make sure the resulting matrix is not too large!
    rdm = PsiAB*PsiAB'
end

nA = 4 # number spins/qubits in A
nB = 5 # number of spins/qubits in B
Psi = randn(2^(nA+nB)) + im*randn(2^(nA+nB))
Psi = Psi/sqrt(Psi'*Psi) 
PsiAB = reshape(Psi, (2^(nA),2^(nB)))
rdmA = compute_rdmA(PsiAB)
eigvals(rdmA)


### 2) Given $\rho_A$, compute its entropy

In [ ]:
function compute_entropy(dm)
    p = eigvals(dm)
    entropy = 0.0
    #entropy = - p'*log2.(p) # problems when p[n] is of order 0 (possibly negative)
    for n in 1:size(p,1)
        if abs(p[n]) > 1e-12
            entropy = entropy - p[n]*log2(p[n])
        end
    end
    entropy
end

compute_entropy(rdmA)

### 3) write a function compute_entropies that takes a state $|\Psi\rangle$ of $N$ spins, and outputs the entropy of the reduced density matrix on $n=1,2,\cdots,N/2$ spins 

In [ ]:
function compute_entropies(Psi)
    @assert ndims(Psi) == 1 
    N = floor(Int64,log2(size(Psi,1) + 1e-5))    
#  your code comes here
    entropies
end


### 4) for $N$ spins ($N \geq 12$) create ground state of Ising model (for $\theta = \pi/2$ and $\theta = \pi/3$), as well as a random state

In [ ]:
# Here is buildH from previous lectures
function buildH(N::Int64,theta::Float64)::Array{Float64,2} 
    X = [0. 1; 1 0]
    Z = [1. 0; 0 -1]
    E = diagm(0=>ones(2))
    XX = kron(X,X)
    HXX = XX
    HZ = kron(Z,E) + kron(E,Z)
    for n = 3:N
        HXX = kron(HXX,E)+kron(diagm(0=>ones(2^(n-2))), XX)
        HZ =  kron(HZ,E) + kron(diagm(0=>ones(2^(n-1))),Z)
    end
    HXX = HXX + kron(X,kron(diagm(0=>ones(2^(N-2))),X))
    H = -cos(theta)*HXX - sin(theta)*HZ
    return H
end

### 5) Both for the ground state (with $\theta = \pi/2$ and $\theta = \pi/3$) and a random state, compute and plot the entropy as a function of $n_A=1,2,\cdots N/2$, 